In [29]:
import pandas as pd

# Load the CSV file
file_path = "Data_cleaned.csv"
df = pd.read_csv(file_path)

# Display basic info and the first few rows
df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Text            1026 non-null   object
 1   Subreddit       1026 non-null   object
 2   Category        1026 non-null   object
 3   TextLength      1026 non-null   int64 
 4   Category_Label  1026 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 40.2+ KB


(None,
                                                 Text  Subreddit  \
 0  gambling addiction disease isnt really happene...     soccer   
 1                         messi last saw still squad     soccer   
 2        ive never heard guess im following hivemind  teenagers   
 3  fugazi narrative one early game huge chance as...     soccer   
 4  self confidence self confidence hard still rel...  AskReddit   
 
           Category  TextLength  Category_Label  
 0         No Slang          19               4  
 1         No Slang           5               4  
 2   Internet Slang           7               1  
 3  Offensive Slang          14               3  
 4         No Slang          48               4  )

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=2)


# Fit and transform the processed text
tfidf_matrix = tfidf_vectorizer.fit_transform(df["Text"])

# Convert the result into a DataFrame for inspection (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Show the TF-IDF result
tfidf_df.head()


,aaa,aaa titles,able,able get,able see,absolute,absolutely,absolutely everywhere,abusive,ac,...,youre getting,youre saying,youre talking,youtube,youve,yup,zelensky,zelenskyy,zero,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# def filter_tokens(tokens):
#     return [t for t in tokens if len(t) > 2]  # remove "aa", "uh", etc.

# df["Tokens"] = df["Tokens"].apply(filter_tokens)

# # Join tokens back into text
# df["Processed_Text"] = df["Tokens"].apply(lambda tokens: " ".join(tokens))

# # Initialize TF-IDF Vectorizer
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# # Fit and transform the processed text
# tfidf_matrix = tfidf_vectorizer.fit_transform(df["Text"])

# # Convert to DataFrame (optional, for inspection or debugging)
# tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# # Show result
# tfidf_df.head()

In [32]:
# Define a custom mapping for categories
category_mapping = {
    "Casual Slang": 0,
    "Internet Slang": 1,
    "Abbreviation": 2,
    "Offensive Slang": 3,
    "No Slang": 4
}

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Features and labels
X = tfidf_matrix
y = df["Category_Label"]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Define the individual models
logreg_model = LogisticRegression(max_iter=1000, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Create the VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('logreg', logreg_model),
    ('rf', rf_model),
    ('svm', svm_model)
], voting='hard')  # 'hard' voting means class with most votes wins

# Train the VotingClassifier
ensemble_model.fit(X_train, y_train)

# Predict on test set
y_pred_ensemble = ensemble_model.predict(X_test)

# Evaluate the model
print("Ensemble Model Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("\nEnsemble Model Classification Report:\n", classification_report(y_test, y_pred_ensemble, target_names=category_mapping.keys()))


Ensemble Model Accuracy: 0.6844660194174758

Ensemble Model Classification Report:
                  precision    recall  f1-score   support

   Casual Slang       0.00      0.00      0.00        13
 Internet Slang       0.57      0.15      0.24        26
   Abbreviation       0.00      0.00      0.00         7
Offensive Slang       1.00      0.04      0.08        23
       No Slang       0.69      0.99      0.81       137

       accuracy                           0.68       206
      macro avg       0.45      0.24      0.23       206
   weighted avg       0.64      0.68      0.58       206



c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
from sklearn.linear_model import LogisticRegression


# Initialize the model
model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=category_mapping.keys()))


Accuracy: 0.6699029126213593

Classification Report:
                  precision    recall  f1-score   support

   Casual Slang       0.00      0.00      0.00        13
 Internet Slang       0.50      0.04      0.07        26
   Abbreviation       0.00      0.00      0.00         7
Offensive Slang       0.00      0.00      0.00        23
       No Slang       0.67      1.00      0.80       137

       accuracy                           0.67       206
      macro avg       0.23      0.21      0.17       206
   weighted avg       0.51      0.67      0.54       206



c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
